In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# model_name = "rmcc11/arth_default_qwen3B-actor-latest"
model_name = "Qwen/Qwen2.5-3B"

# Initialize model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

/workspace/venvs/.tiny_zero/lib/python3.10/site-packages/huggingface_hub/file_download.py:651: UserWarning: Not enough free disk space to download the file. The expected file size is: 3968.66 MB. The target location /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-3B/blobs only has 3374.69 MB free disk space.
  warnings.warn(


model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

In [20]:
num1 = "323"
op = "*"
num2 = "68"

initial_prompt_standard = f"""A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> RESULT_NUMBER </answer>. \nUser: Give me the answer of the following equation: {num1} {op} {num2}.\nAssistant: Let me solve this step by step.\n<think>"""

initial_prompt_replace = f"""A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. When performing calculations, the assistant replaces mathematical symbols with words: '*' -> 'by', '-' -> 'minus', '+' -> 'plus', and '=' -> 'equals'. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> RESULT_NUMBER </answer>. \nUser: Give me the answer of the following equation: {num1} {op} {num2}.\nAssistant: Let me solve this step by step.\n<think>"""

initial_prompt_replace_vague = f"""A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. When performing calculations, the assistant replaces mathematical symbols (e.g., '*', '-', '+', '=') with something else. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> RESULT_NUMBER </answer>. \nUser: Give me the answer of the following equation: {num1} {op} {num2}.\nAssistant: Let me solve this step by step.\n<think>"""

completion = "To find the product of 323 and 68, we can use the distributive property: 323 * 68 = 323 * (60 + 8) = (323 * 60) + (323 "
completion_initial = "To find the product of 323 and 68, we can use the distributive property: 323 "

In [4]:
def get_n_single_tokens(prompt: str, n: int, temperature: float = 0.7) -> list:
    """Generate N single tokens from a prompt using specified temperature."""
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    generated_tokens = []
    
    for _ in range(n):
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1,
                do_sample=True,
                temperature=temperature,
                pad_token_id=tokenizer.pad_token_id,
                num_return_sequences=1
            )
            
            new_token = outputs[0][len(inputs['input_ids'][0]):]
            decoded_token = tokenizer.decode(new_token)
            generated_tokens.append(decoded_token)
    
    return generated_tokens

In [22]:
def get_n_single_tokens_batched(prompt: str, n: int, batch_size: int = 32, temperature: float = 0.7) -> list:
    """Generate N single tokens from a prompt using specified temperature and batch processing.
    
    Args:
        prompt: Input prompt to generate tokens from
        n: Number of tokens to generate
        batch_size: Number of tokens to generate in parallel
        temperature: Sampling temperature
        
    Returns:
        List of generated tokens as strings
    """
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    generated_tokens = []
    
    # Calculate number of full batches and remaining tokens
    num_full_batches = n // batch_size
    remaining_tokens = n % batch_size
    
    # Process full batches
    for _ in range(num_full_batches):
        with torch.no_grad():
            # Expand input tensors to match batch size
            batch_inputs = {
                k: v.expand(batch_size, -1) for k, v in inputs.items()
            }
            
            outputs = model.generate(
                **batch_inputs,
                max_new_tokens=1,
                do_sample=True,
                temperature=temperature,
                pad_token_id=tokenizer.pad_token_id,
                num_return_sequences=batch_size
            )
            
            # Extract and decode new tokens from each sequence
            for seq in outputs:
                new_token = seq[len(inputs['input_ids'][0]):]
                decoded_token = tokenizer.decode(new_token)
                generated_tokens.append(decoded_token)
    
    # Process remaining tokens if any
    if remaining_tokens > 0:
        with torch.no_grad():
            batch_inputs = {
                k: v.expand(remaining_tokens, -1) for k, v in inputs.items()
            }
            
            outputs = model.generate(
                **batch_inputs,
                max_new_tokens=1,
                do_sample=True,
                temperature=temperature,
                pad_token_id=tokenizer.pad_token_id,
                num_return_sequences=remaining_tokens
            )
            
            for seq in outputs:
                new_token = seq[len(inputs['input_ids'][0]):]
                decoded_token = tokenizer.decode(new_token)
                generated_tokens.append(decoded_token)
    
    return generated_tokens

In [ ]:
prompt = initial_prompt_standard + completion
n_generations = 200
temperature = 1.0
batch_size = 4

# Example usage
tokens = get_n_single_tokens(prompt, n=n_generations, temperature=temperature)
# tokens = get_n_single_tokens_batched(prompt, n=n_generations, temperature=temperature, batch_size=batch_size)
print("Generated tokens:")
for token in tokens:
    print(token)

Generated tokens:
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
6
 *
 *
 *
 *
6
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
6
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
6
6
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
6
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
6
 *
 *
6
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
 *
6
 *
 *
 *
 *


In [32]:
valid_strings_to_check_for = ["*", "x", "by", "time", "mul"]

# Count tokens containing valid strings
token_counts = {s: sum(1 for t in tokens if s in t) for s in valid_strings_to_check_for}
total_valid_tokens = sum(1 for t in tokens if any(s in t for s in valid_strings_to_check_for))

print(f"Temperature: {temperature}")
print(f"Total tokens generated: {len(tokens)}")
print(f"Number of tokens containing valid strings: {total_valid_tokens}")
print("\nCounts per valid string:")
for string, count in token_counts.items():
    print(f"'{string}': {count}")

# Count all unique tokens and get top 5 by occurrence
from collections import Counter
token_counter = Counter(tokens)
print("\nTop 5 tokens by occurrence:")
for token, count in token_counter.most_common(5):
    print(f"'{token}': {count}")



Temperature: 1.0
Total tokens generated: 200
Number of tokens containing valid strings: 191

Counts per valid string:
'*': 191
'x': 0
'by': 0
'time': 0
'mul': 0

Top 5 tokens by occurrence:
' *': 191
'6': 9
